In [ ]:
import glob
import os
from skimage import io, img_as_float, color
from skimage.util import random_noise
import numpy as np
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Input
from tensorflow.keras.models import Model

# Load and preprocess dataset
def load_images_from_paths(paths):
    images = []
    for img_path in paths:
        img = img_as_float(io.imread(img_path))
        if img is not None:
            gray_img = color.rgb2gray(img)
            images.append(gray_img)
    return np.array(images)

def add_noise(images):
    noisy_images = []
    for img in images:
        noisy_img = random_noise(img, mode='gaussian', var=0.01)
        noisy_images.append(noisy_img)
    return np.array(noisy_images)

# Define DnCNN model
def build_dncnn_model():
    input_img = Input(shape=(None, None, 1))
    x = Conv2D(64, (3, 3), padding='same')(input_img)
    x = Activation('relu')(x)
    for _ in range(15):
        x = Conv2D(64, (3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    x = Conv2D(1, (3, 3), padding='same')(x)
    model = Model(inputs=input_img, outputs=x)
    return model

# Get image paths
vehicle_paths = sorted(glob.glob('./VehicleImage/vehicles/*/*.png'))
nonvehicle_paths = sorted(glob.glob('./VehicleImage/non-vehicles/*/*.png'))

# Load images
vehicle_images = load_images_from_paths(vehicle_paths)
nonvehicle_images = load_images_from_paths(nonvehicle_paths)

# Combine and add noise to images
all_images = np.concatenate((vehicle_images, nonvehicle_images), axis=0)
all_images_noisy = add_noise(all_images)

# Prepare training data
x_train = np.expand_dims(all_images_noisy, axis=-1)
y_train = np.expand_dims(all_images, axis=-1)

# Build and compile model
model = build_dncnn_model()
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

# Save the model
model.save('dncnn_model.h5')

Epoch 1/50


2024-10-24 16:56:36.569807: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


412/412 ━━━━━━━━━━━━━━━━━━━━ 184s 417ms/step - loss: 0.0511 - val_loss: 0.0100
Epoch 2/50
165/412 ━━━━━━━━━━━━━━━━━━━━ 1:59 486ms/step - loss: 0.0038